In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq
 
from bokeh.palettes import brewer, Spectral6
from bokeh.io import show, output_file, curdoc
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, Panel, Tabs, GeoJSONDataSource, LinearColorMapper
from bokeh.models import ColorBar
from bokeh.layouts import column, gridplot, row
from bokeh.transform import factor_cmap
from bokeh.models import NumeralTickFormatter, Title, Label, Paragraph, Div, CustomJSHover, BoxAnnotation

This notebook grabs lobster exports and then illustrates what is going on with them

In [2]:
trade_type = "exports"

my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"
# This is my key. I'm nice and I have it posted. If you will be doing more with this
# please get your own key!

In [3]:
crl = ["darkblue","slategray","crimson"]

background = "#ffffff"

In [4]:
last_month = 12

---
#### Grabe the trade data using the Census's API

In [5]:
def census_trade(url, trade_type, product_level):
    
    r = requests.get(url) 
    
    print(r)
    
    df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe
    # Note that the first entry is the labels

    df.columns = r.json()[0]

    df.time = pd.to_datetime(df.time, format="%Y-%m")
    # This is so I can call this correctly...
    
    if trade_type == "imports":
        
        trade_label = trade_type
        
        df[trade_label] = df["CON_VAL_MO"].astype(float)
        
        df[product_level] = df["I_COMMODITY"].astype(str)
        
        df.drop(["CON_VAL_MO", "I_COMMODITY", "COMM_LVL"], axis = 1, inplace = True)
        
    if trade_type == "exports":
    
        trade_label = trade_type
        
        df[trade_label] = df["ALL_VAL_MO"].astype(float)

        df[product_level] = df["E_COMMODITY"].astype(str)        
        
        df.drop(["ALL_VAL_MO", "E_COMMODITY", "CTY_CODE", "COMM_LVL"], axis = 1, inplace = True)
    
    return df

In [6]:
def growth_trade(foo):
    # what this function does is take a dataframe and create a relative 
    
    foo["growth"] = 100*((foo.exports/foo.exports.shift(12)) - 1)
    
    return foo

---
### This is the Product Specific plot...

In [7]:
end_use = "hs?get=ALL_VAL_MO,CTY_CODE,CTY_NAME"

url = "https://api.census.gov/data/timeseries/intltrade/exports/" + end_use 
url = url + my_key + "&time==from+2013-01" + "&COMM_LVL=HS4" + "&E_COMMODITY=0306"

url = url

url = url

df = census_trade(url,"exports","hs4")

<Response [200]>


In [8]:
df.CTY_NAME.unique()

array(['TOTAL FOR ALL COUNTRIES', 'GREENLAND', 'CANADA', 'MEXICO',
       'GUATEMALA', 'BELIZE', 'EL SALVADOR', 'HONDURAS', 'NICARAGUA',
       'COSTA RICA', 'PANAMA', 'BERMUDA', 'BAHAMAS', 'JAMAICA',
       'TURKS AND CAICOS ISLANDS', 'CAYMAN ISLANDS', 'HAITI',
       'DOMINICAN REPUBLIC', 'ANGUILLA', 'BRITISH VIRGIN ISLANDS',
       'ST KITTS AND NEVIS', 'ANTIGUA AND BARBUDA', 'MONTSERRAT',
       'DOMINICA', 'ST LUCIA', 'ST VINCENT AND THE GRENADINES', 'GRENADA',
       'BARBADOS', 'TRINIDAD AND TOBAGO', 'SINT MAARTEN', 'CURACAO',
       'ARUBA', 'GUADELOUPE', 'MARTINIQUE', 'COLOMBIA', 'VENEZUELA',
       'GUYANA', 'SURINAME', 'FRENCH GUIANA', 'ECUADOR', 'PERU', 'CHILE',
       'BRAZIL', 'URUGUAY', 'ARGENTINA', 'ICELAND', 'SWEDEN',
       'SVALBARD, JAN MAYEN ISLAND', 'NORWAY', 'FINLAND', 'DENMARK',
       'UNITED KINGDOM', 'IRELAND', 'NETHERLANDS', 'BELGIUM', 'FRANCE',
       'GERMANY', 'AUSTRIA', 'CZECH REPUBLIC', 'HUNGARY', 'SWITZERLAND',
       'ESTONIA', 'LATVIA', 'LITHUANIA', 

In [9]:
df.head()

,CTY_NAME,time,exports,hs4
0,TOTAL FOR ALL COUNTRIES,2013-01-01,58109297.0,0306
1,TOTAL FOR ALL COUNTRIES,2013-02-01,67784422.0,0306
2,TOTAL FOR ALL COUNTRIES,2013-03-01,78140862.0,0306
3,TOTAL FOR ALL COUNTRIES,2013-04-01,52701883.0,0306
4,TOTAL FOR ALL COUNTRIES,2013-05-01,43229949.0,0306


In [10]:
df = df[["CTY_NAME","time","exports"]]

In [11]:
grp = df.groupby(["CTY_NAME"])

top_products = grp.agg({"exports":"sum"})

#clist = list(top_products.sort_values(by = "total_imports", ascending = False).index.values)[0:5]

clist = ["TOTAL FOR ALL COUNTRIES", "CHINA", "EUROPEAN UNION"]

df = df[df.CTY_NAME.isin(clist)]

In [12]:
grp = df.groupby(["CTY_NAME"])

country_df = grp.apply(growth_trade)

In [13]:
def make_trade_time_country(df):
        
    foo = df.reset_index().pivot(index = "time", columns = "CTY_NAME")
    
    foo.drop(labels = ["index"], axis = 1, inplace = True)
    
    return foo

In [14]:
test = make_trade_time_country(country_df)

In [15]:
test.head()

exports                                        growth  \
CTY_NAME         CHINA EUROPEAN UNION TOTAL FOR ALL COUNTRIES  CHINA   
time                                                                   
2013-01-01   9662748.0     10119033.0              58109297.0    NaN   
2013-02-01  22255301.0     12066452.0              67784422.0    NaN   
2013-03-01  29596815.0      9273155.0              78140862.0    NaN   
2013-04-01  17381308.0      8725589.0              52701883.0    NaN   
2013-05-01   7842250.0     13269168.0              43229949.0    NaN   

                                                   
CTY_NAME   EUROPEAN UNION TOTAL FOR ALL COUNTRIES  
time                                               
2013-01-01            NaN                     NaN  
2013-02-01            NaN                     NaN  
2013-03-01            NaN                     NaN  
2013-04-01            NaN                     NaN  
2013-05-01            NaN                     NaN

In [16]:
def make_time_by_country(df, level):

    height = 533
    width = 600

    foobar = make_trade_time_country(df)
    
    if level == "exports":
        title = "$ US Exports of Lobsters (HS 0306), China, EU, Total"
        
        #foobar.drop([ "TOTAL FOR ALL COUNTRIES"], axis = 1, level = 1, inplace = True)
    
    if level == "growth":
        title = "% YoY Change in US Exports of Lobsters (HS 0306), China, EU, Total"
    
    p = figure(plot_height=height, plot_width = width, x_axis_type="datetime",toolbar_location = 'below',
           tools = "box_zoom, reset, pan, xwheel_zoom,ywheel_zoom", title = title, x_range = (dt.datetime(2015,1,1),dt.datetime(2020,last_month,1)) ) 

    numlines=len(foobar[level].columns)

    multi_line_source = ColumnDataSource({
        'xs': [foobar.index.values]*numlines,
        'ys': [foobar[level, name].values for name in foobar[level]],
        'label': [name for name in foobar["exports"].columns],
        'color': ["crimson", "darkblue", "slategrey"]})

    p.multi_line(xs= "xs",
                ys= "ys",
                line_width=2.5, line_alpha=0.45, line_color = "color",
                 hover_line_alpha=0.75, hover_line_width = 5,
                hover_line_color= "color", source = multi_line_source)
    
    y_custom = CustomJSHover(code=""" return '' + special_vars.data_y
            """)
            
    TIMETOOLTIPS = """
            <div style="background-color:#F5F5F5; opacity: 0.95; border: 5px 5px 5px 5px;">
            <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold"> @label
             </span>
             </div>
             <div style = "text-align:left;">"""
    
    if level == "exports":
        TIMETOOLTIPS = TIMETOOLTIPS + """
            <span style="font-size: 13px; font-weight: bold"> $data_x{%b %Y}:  $data_y{$0.0a}</span>   
            </div>
            </div>
            """
    if level == "growth":
        TIMETOOLTIPS = TIMETOOLTIPS + """
            <span style="font-size: 13px; font-weight: bold"> $data_x{%b %Y}:  $data_y{0.0}%</span>   
            </div>
            </div>
            """

    p.add_tools(HoverTool(tooltips = TIMETOOLTIPS,  line_policy='nearest', formatters={'$data_x': 'datetime'}))
    p.title.text_font_size = '13pt'
    p.background_fill_color = background 
    p.background_fill_alpha = 0.75
    p.border_fill_color = background 
    
    
    tradewar_box = BoxAnnotation(left=dt.datetime(2018,7,1), right=dt.datetime(2019,10,11), fill_color='slategrey', fill_alpha=0.1)
    p.add_layout(tradewar_box)
    
    tradewar_box = BoxAnnotation(left=dt.datetime(2020,1,1), right=dt.datetime(2020,12,31), fill_color='blue', fill_alpha=0.1)
    p.add_layout(tradewar_box)
    
    #p.yaxis.axis_label = 
    p.yaxis.axis_label_text_font_style = 'bold'
    p.yaxis.axis_label_text_font_size = "13px"

    p.yaxis.minor_tick_line_color = None
    
    if level == "exports":
        p.yaxis.formatter = NumeralTickFormatter(format="($0. a)")
        p.y_range.end = 45000000
        
    if level == "growth":
        p.y_range.end = 125
    
    p.outline_line_color = None
    p.sizing_mode= "scale_both"
    p.max_height = height
    p.max_width = width
    
    p.toolbar.active_drag = None
    p.toolbar.autohide = True
    p.min_border_left = 0
    p.min_border_bottom = 0

    return p

In [17]:
clevel = make_time_by_country(country_df, "exports")
    
cchange = make_time_by_country(country_df, "growth")

tab3 = Panel(child= clevel, title="Imports in $")

tab4 = Panel(child= cchange, title="Imports as % YoY")

output_file('.\\docs\\' + "us_exports_lobster.html")

div0 = Div(text = """Shaded grey area indicates when the Section 301 tariffs on China were in place and retaliation.
Blue is period of agreement. Hover over the line with your curser to view month by month values.""", max_width=555, background = background )
div0.sizing_mode= "scale_both"
        
outfig = column(Tabs(tabs=[tab3,  tab4], tabs_location = "above"), div0, sizing_mode="scale_both")

show(outfig)

In [18]:
country_df.CTY_NAME.unique()

array(['TOTAL FOR ALL COUNTRIES', 'CHINA', 'EUROPEAN UNION'], dtype=object)